## 创建环境

TorchRL不直接提供环境，而是提供了经典环境的包装
可以按照以下的方式创建环境

In [2]:
from torchrl.envs import GymEnv

env = GymEnv("Pendulum-v1")

# 运行环境

TorchRL中的环境有两个重要的方法，一个是reset()初始化一个episode,另一个是step(),执行一个由actor选择的动作。
在TorchRL中环境的方法读和写都是TensorDict实例。TensorDict本质上是一个基于关键字的数据载体，像python字典一样。
使用TensorDict的好处是，相比普通的Tensor，他赋予我们对复杂数据结构的更大的灵活性。
下面是TensorDict使用的示例

In [3]:
reset = env.reset()
print(reset)

TensorDict(
    fields={
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        observation: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.float32, is_shared=False),
        terminated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        truncated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False)},
    batch_size=torch.Size([]),
    device=cpu,
    is_shared=False)


In [4]:
reset_with_action = env.rand_action(reset)
print(reset_with_action)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        observation: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.float32, is_shared=False),
        terminated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        truncated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False)},
    batch_size=torch.Size([]),
    device=cpu,
    is_shared=False)


通过上面两个例子可以看出，所有的tensordict都有的相同的结构，不同的是随机选择一个动作后返回的实例多了一个action实体
并且可以通过访问普通字典那样获取他的值

In [5]:
print(reset_with_action["action"])

tensor([0.6153])


现在我们需要给环境传入一个动作，不同的是，现在传入的是一个整个tensordict实例。
为什么传入整个tensordict实例，因为在更高级的强化学习方法中，例如多智能体强化学习中环境的读取的变量可能更多。

In [6]:
stepped_data = env.step(reset_with_action)
print(stepped_data)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([]),
            device=cpu,
            is_shared=False),
        observation: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.float32, is_shared=False),
        terminate

step()执行一个动作之后返回的仍然是一个TensorDict，不同的是多了一个'next'实体，'next'实体他本身也是一个tensordict，包含了observation，reward，done state。

这种格式叫做TED，TorchRL Episode Data format。

下一件事在运行一个完成rollout需要了解的就是如何用'next'实体去执行下一个step，
TorchRL定制了一个 step_mdp()函数实现这个功能：过滤掉哪些在执行一个MDP决策后你不需要的信息，

例如，在下面的例子中stepped_data，data就只剩下了'next'信息

In [7]:
from torchrl.envs import step_mdp

data = step_mdp(stepped_data)
print(data)

TensorDict(
    fields={
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        observation: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.float32, is_shared=False),
        terminated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        truncated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False)},
    batch_size=torch.Size([]),
    device=cpu,
    is_shared=False)


环境展开（rollout）
反复写以下三步（选择动作，执行动作，moving in MDP）很繁琐，TorchRL提供了一个rollout()函数，在一个封闭的循环中执行这3步

In [8]:
rollout = env.rollout(max_steps=10)
print(rollout)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([10, 3]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([10]),
            device=cpu,
            is_shared=False),
        observation: Tensor(shape=torch.Size([10, 3]), device=cpu, dtype=torch.float32, i

上面这个数据的输出，和之前的stepped_data几乎一样，除了batch_size外，而恰巧batch_size等于max_step参数。
torchDict的神奇之处远不只此，如果你对一个单独transition感兴趣，可以通过索引去访问他。
TensorDict会自动检测索引是一个关键字kay还是一个空间地址。

In [9]:
transition = rollout[3]
print(transition)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([]),
            device=cpu,
            is_shared=False),
        observation: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.float32, is_shared=False),
        terminate

像这样执行（没有策略），rollout方法似乎相当无用：它只是随机地运行动作。
如果有可用的策略，它可以传递给该方法并用来收集数据。
然而，首先进行一个简单的、无策略的rollout可能会很有用，以便一眼就可以看出预期从环境中得到什么。
要欣赏TorchRL API的多功能性，考虑一下这样一个事实：rollout方法具有普遍适用性。
它在所有使用场景都能工作，无论你是在像这样的单一环境中工作，还是在多个进程中的多个副本，或者是多代理环境，甚至是它的无状态版本！

大多数时候，你可能希望修改环境的输出以更好地适应你的需求。例如，你可能想获取自上次重置（reset）以来执行的步数，调整图像大小，或将连续的观察结果堆叠在一起。
在这一部分，我们将研究一个简单的变换（transform），即StepCounter变换。完整的变换列表可以在这里找到。
通过TransformedEnv，变换与环境进行了集成：

In [10]:
from torchrl.envs import StepCounter, TransformedEnv

transformed_env = TransformedEnv(env, StepCounter(max_steps=10))
rollout = transformed_env.rollout(max_steps=100)
print(rollout)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([10, 3]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                step_count: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.int64, is_shared=False),
                terminated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([10]),
            device=cpu,
        

如你所见，我们的环境现在多了一项"step_count"，这项数据用来追踪自上次重置（reset）以来走了多少步。
考虑到我们将可选参数max_steps=10传递给了变换构造函数，我们还在10步后截断了轨迹（没有像我们用rollout调用请求的那样完成全部100步）。

In [11]:
print(rollout["next", "truncated"])

tensor([[False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True]])


In [12]:
print(rollout['next']) # 返回了10个next

TensorDict(
    fields={
        done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        observation: Tensor(shape=torch.Size([10, 3]), device=cpu, dtype=torch.float32, is_shared=False),
        reward: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        step_count: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.int64, is_shared=False),
        terminated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        truncated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
    batch_size=torch.Size([10]),
    device=cpu,
    is_shared=False)


In [13]:
print(rollout['next','step_count'])

tensor([[ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10]])


In [14]:
print(rollout['next']['step_count'])

tensor([[ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10]])
